In [9]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\util|s_zonca\\pointing')
sys.path.append('../utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../VtoT')
print os.getcwd()
import convert
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
import pandas as pd
#from pointingtools import compute_parallactic_angle, altaz2ha 
from planets import getlocation, getpointing
import warnings
from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
from astropy import units as u
import ephem
import matplotlib.pyplot as plt
import time
import planets
from datetime import timedelta
import numpy.ma as ma

import Tkinter,tkFileDialog

from Tkinter import *
import ttk

import pickle
import glob

import cPickle

%pylab


D:\software_git_repos\Polaris\polaris_software\notebook
Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\users\labuser\anaconda3\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
def get_pointing_files(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/polaris/polaris_data/pointing_data',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    return filelist

In [3]:
def read_some_data(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/polaris/polaris_data/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    dlist=[]
    for f in filelist:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=np.concatenate(dlist)
    hf.close() 

    datadict=d
        
    return datadict, filelist

In [4]:
def get_file_times(fld):
    startfile = fld[0][:65]+fld[0][71:-2]+'dat'
    endfile = fld[-1][:65]+fld[-1][71:-2]+'dat'
    
    #starttime = os.path.getctime(startfile)
    starttime= os.stat(startfile).st_mtime
    starttime = datetime.datetime.fromtimestamp(starttime)

    #endtime = os.path.getctime(endfile)
    endtime= os.stat(endfile).st_mtime
    endtime = datetime.datetime.fromtimestamp(endtime)
    
    return starttime, endtime

In [5]:
def plotnow_azrevsig(data, az, chan, var, res = 1.0, jump = 180.,minmax=None,supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=rt.get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=rt.get_all_demodulated_data(fld_demod, fld)
    #combined=rt.combine_cofe_h5_pointing(dd,pp)

    #synchronized data and az values
    az1 = az
    data1 = data
    steps = len(data1)

    #convert to temp for cryo sensors
    if chan == 12:
        data1 = convert.convert(data1, 'i')
    if chan == 13:
        data1 = convert.convert(data1, 'e')
    if chan == 14:
        data1 = convert.convert(data1, 'h')
    if chan == 15:
        data1 = convert.convert(data1, 'l')

    #resolution
    dx = res
    dy = 1.0

    #set up empty lists to append each revolution to
    data = []
    az = []
    iaz = [0]
    rev = 0

    #determine indices in azimuth/data array which correspond to a new revolution of the telescope
    for i in range(steps):
        #round values to resolution for comparison later
        az1[i] = rt.round_fraction(az1[i], dx)
        if i > 0:
            if abs(az1[i] - az1[i-1]) >= jump:
                iaz.append(i)
                rev += 1

    #append each revolution array to a list	    
    for j in range(rev):
        az.append(az1[iaz[j]:iaz[j+1]])
        data.append(data1[iaz[j]:iaz[j+1]])

    #append the last revolution
    data.append(data1[iaz[-1]:])
    az.append(az1[iaz[-1]:])
    rev += 1

    print rev

    data = np.asarray(data)
    az = np.asarray(az)

    #create grid for plotting
    x, y = np.arange(0., 360.+dx, dx), np.arange(0., rev - 1 + dy, dy)
    AZ, REV = np.meshgrid(x, y)

    #set up empty array
    z = np.zeros(len(x)*len(y))
    sig = np.reshape(z, (len(y), len(x)))

    #small number for comparing floats
    epsilon = 1e-6

    #fill signal array with data points
    for r in range(rev):
        for a in range(len(x)):
            #find indices where combined azimuth data fits on x grid
            idx = np.where(abs(az[r] - x[a]) < epsilon)[0]
            #if idx length is 0 this will create a mask on that point, in idx len > 1, avg data points in the same bin
            sig[r][a] = data[r][idx].mean()

    #mask invalid values, i.e. where there are no data points
    sig = ma.masked_invalid(sig)

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)

    plt.pcolormesh(AZ, REV, sig)
    plt.colorbar(label = 'Signal, %s' % unit)
    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data1.min(),data1.max())    #plt.clim(-1.0,1.0)
    plt.axis([0., 360., 0., rev - 1])
    plt.ylabel('revolution #')
    plt.xlabel('azimuth (deg)')
    plt.title('%s %s data binned to azimuth and revolution #' % (name, var))
    plt.grid()
    plt.show()

In [6]:
def plotnow_azelsig(data, az, el, chan, var, res, minmax=None, radec=False, supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)	
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data az and el values
    az1, el1 = az, el
    data = data

    #convert to temp for cryo sensors
    if chan == 12:
        data = convert.convert(data, 'i')
    if chan == 13:
        data = convert.convert(data, 'e')
    if chan == 14:
        data = convert.convert(data, 'h')
    if chan == 15:
        data = convert.convert(data, 'l')

    steps = len(data)

    #set az/el resolution
    dx = res
    dy = res

    #set up bins/grid
    if radec:
        x, y = np.arange(0., 360.+dx, dx), np.arange(-90., 90. + dy, dy)
    else:
        x, y = np.arange(0., 360.+dx, dx), np.arange(0., 90. + dy, dy)
    AZ, EL = np.meshgrid(x, y)

    #small number for comparing floats
    epsilon = 1e-6

    #set up matrix for signal 
    z1 = np.zeros(len(x)*len(y))
    sig = np.reshape(z1, (len(y), len(x)))

    #set up matrix for keeping track of data points in single bin for averaging
    z2 = np.zeros(len(x)*len(y))
    count = np.reshape(z2, (len(y), len(x)))

    for i in range(steps):

        #round az/el points for comparison with grid	    
        el1[i] = rt.round_fraction(el1[i], dy)
        az1[i] = rt.round_fraction(az1[i], dx)  

        #find where data points belong in grid
        iel = np.where(abs(y - el1[i]) < epsilon)[0][0]
        iaz = np.where(abs(x - az1[i]) < epsilon)[0][0]

        #add 1 each time data point lands in same bin
        count[iel][iaz] += 1

        #add total number of data values in bin
        sig[iel][iaz] = sig[iel][iaz] + data[i]  

    #mask 0 count values so they dont show up in color plot
    count = ma.masked_where(count == 0.0, count)

    #take average of all data points in single bin
    sig = sig/count
    
    try:
        #change units on plot label
        if int(chan[2:]) < 12:
            unit = 'V'
        else:
            unit = 'K' 
            
        name = rt.chantoname(chan)
    except:
        unit = 'C'
        name = chan

    

    plt.pcolormesh(AZ, EL, sig)
    plt.colorbar(label = 'Signal, %s' % unit)
    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data.min(),data.max())
    if radec == False:
        plt.axis([AZ.min(), AZ.max(), EL.min(), EL.max()])
        plt.ylabel('elevation (deg)')
        plt.xlabel('azimuth (deg)')
    else:
        plt.axis([0., 360., -90, 90.])
        plt.xlabel('ra (deg)')
        plt.ylabel('dec (deg)')
    plt.title('%s %s data binned to azimuth and elevation' % (name, var))
    plt.grid()
    plt.show()

In [7]:
def get_h5_pointing_gps(filelist,startrev=None, stoprev=None,angles_in_ints=False,azel_era=3):
    """
        modify to fast version, dont' average the multiple values, just take the first one. will cause a little bias but its very fast.
    also implmented removal of erroneous endof h5 file crap, and az outliers
    azel_era determines what az/el offsets to use, 1 means before 9/26/2013, 2 means 9/26-10/3 (inclusive), 3 means after 10/4.
    """



    hpointing=[]
    filelist.sort()

    for f in filelist:
        '''
        if f[79:88] != 'corrected':
            slimit = 22144
        else:
            slimit = 11512
        stats=os.stat(f)
        if stats.st_size<slimit:
            print f,stats.st_size
        if stats.st_size >= slimit:
        '''
        h=h5py.File(f)
        hh=h['data']
        hpointing.append(hh[:])
        #print hpointinng
        h.close()

    hpointing = np.concatenate(hpointing)
    #cut out blank lines from unfilled files
    if startrev != None:
        hpointing=hpointing[hpointing['gpstime'] > startrev]
    if stoprev != None:
        hpointing=hpointing[hpointing['gpstime'] < stoprev]

    hrevlist,inds=np.unique(hpointing['gpstime'],return_index=True)


    #gpslock=hpointing['gpslock']#[inds]
    
    return {'gpstime':hrevlist}

In [8]:
ddict={}

In [10]:
dd, fld = read_some_data()

IOError: Unable to create file (Unable to open file: name = 'e:/software_git_repos/polaris/polaris_data/data/20220914/17594800.dat', errno = 17, error message = 'file exists', flags = 15, o_flags = 502)

In [22]:
flp = get_pointing_files()

In [23]:
pp = rt.get_h5_pointing(flp)
#pp = get_h5_pointing_gps(flp)

In [26]:
pp.keys()

['gpstime',
 'el',
 'computer time',
 'el offset',
 'Calibrator',
 'H2',
 'H3',
 'H1',
 'Amplifier',
 'az offset',
 'Cooler',
 'flag',
 'x tilt',
 'Phidget Temp',
 'y tilt',
 'az',
 'Backend TSS']

In [37]:
gpstime = ((pp['gpstime'] - pp['gpstime'][0])/1000./60.)
#flags = pp['flag']

In [38]:
plt.figure()
plt.plot(gpstime)

In [24]:
combined = rt.combine_cofe_h5_pointing(dd, pp)

In [25]:
#lvl1f=h5py.File('D:/software_git_repos/greenpol/telescope_control/data_aquisition/level1/20180510/ch1-10_34_06-15_56_00.h5')['data']

In [26]:
starttime, endtime = rt.get_file_times(fld)
dt , ut= rt.convert_gpstime(starttime, combined['gpstime']/1000, format='datetime')
dt = np.asarray(dt)

In [27]:
az, el = combined['az'], combined['el']

In [28]:
chan = 'H3HiDC'
var = 'T'
data = combined['sci_data'][rt.nametochan(chan)][var]

In [24]:
isort = np.argsort(az)
iaz = az[isort]
idata = data[isort]

In [32]:
gpstime = (combined['gpstime'] - combined['gpstime'][0])/3600

In [31]:
figure()
plotnow_azrevsig(data, az, rt.nametochan(chan), var, 0.5, minmax = [-0.32, -0.37])

60


In [23]:
figure()
plotnow_azelsig(data, az, el, rt.nametochan(chan), var, 0.5, radec=False, minmax = [-0.06, 0.06])

In [123]:
plt.figure()
plt.plot(ut, data)

In [1]:
#azsuntrue, altsuntrue = getpointing('UCSB', 'Sun', datetime.datetime.fromtimestamp(time.time()))
azsuntrue, altsuntrue = getpointing('Sedgwick', 'Sun', datetime.datetime.utcnow())

print altsuntrue, azsuntrue

NameError: name 'getpointing' is not defined

In [149]:
 
location = getlocation('Sedgwick')

t1 = time.time()
#create ra dec sky object
azel = SkyCoord(az = az, alt = el, obstime = dt, location = location, frame = 'altaz', unit='deg')
t2 = time.time()
print t2-t1
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    #convert from ra dec to az/el for pointing
    radec = azel.icrs
print time.time()-t2
ra = radec.ra.deg
dec = radec.dec.deg

0.204999923706
12.9049999714


In [120]:
figure()
plotnow_azelsig(pp['Phidget Temp'], pp['az'], pp['el'], 'Phidget', var, 1, radec=False)

In [157]:
figure()
plotnow_azelsig(data, ra, dec, rt.nametochan(chan), var, 0.5, radec=True, minmax = [-0.5, 0.0])

In [63]:
xa = az.copy()

In [64]:
datas = [x for _,x in sorted(zip(xa,data))]
xa = sorted(xa)

In [124]:
imin = np.where(data==data.min())[0][0]
print imin
print data[imin]
print data.min()

75016
-1.10123515129
-1.10123515129


In [125]:
azsuntrue, altsuntrue = getpointing('Sedgwick', 'Moon', datetime.datetime.fromtimestamp(ut[imin]))


In [126]:
azsuntrue, altsuntrue = getpointing('Sedgwick', 'Moon', datetime.datetime.fromtimestamp(ut[imin]))
azsunfalse, altsunfalse = az[imin], el[imin] 
print 'time at maximum: ', dt[imin]
print 'ephem az, el:', azsuntrue, altsuntrue
print 'current encoder az, el:', azsunfalse, altsunfalse
print 'az offset: ', azsunfalse - azsuntrue
print 'el offset:', altsunfalse - altsuntrue
print 'applied az offset (start/finish): ', pp['az offset'][0], pp['az offset'][-1]

time at maximum:  2020-10-30 03:45:10.235000
ephem az, el: 114.816222222 36.86375
current encoder az, el: 115.0 35.0009846379
az offset:  0.183777777778
el offset: -1.86276536208
applied az offset (start/finish):  272.888 272.888


In [ ]:
'''
time at maximum:  2018-12-04 21:44:07.011000
ephem az, el: 209.8335 27.1236388889
current encoder az, el: 169.0 24.9566064453
az offset:  -40.8335
el offset: -2.16703244358
applied az offset (start/finish):  283.188481657 283.188481657
'''

In [93]:
mi = len(data)/2
ma = 3*len(data)/4
imin = np.where(data[mi:ma]==data[mi:ma].min())[0][0]
print imin
print data[mi:ma][imin]
print data[mi:ma].min()

17417
-5.50108075142
-5.50108075142


In [70]:
#azsuntrue, altsuntrue = getpointing('UCSB', 'Sun', datetime.datetime.fromtimestamp(time.time()))
azsuntrue, altsuntrue = getpointing('Sedgwick', 'Moon', datetime.datetime.utcnow())

print altsuntrue, azsuntrue

34.2717777778 79.6356666667


In [10]:
cra = 15*(5 + (34./60.) + (31.94/3600.))
cdec = 22 + (52.2/3600.)

caz, cel = planets.radec_to_azel(cra, cdec, 'Sedgwick', datetime.datetime.utcnow())

In [11]:
print cdec, cra
print cel, caz

22.0145 83.6330833333
63.2813567275 248.838721759


In [95]:
azsuntrue, altsuntrue = getpointing('Sedgwick', 'Sun', datetime.datetime.fromtimestamp(ut[mi:ma][imin]))
azsunfalse, altsunfalse = az[mi:ma][imin], el[mi:ma][imin] 
print 'time at maximum: ', dt[mi:ma][imin]
print 'ephem az, el:', azsuntrue, altsuntrue
print 'current encoder az, el:', azsunfalse, altsunfalse
print 'az offset: ', azsunfalse - azsuntrue
print 'el offset:', altsunfalse - altsuntrue
print 'applied az offset (start/finish): ', pp['az offset'][0], pp['az offset'][-1]

time at maximum:  2018-07-31 14:29:38.845000
ephem az, el: 176.911416667 35.5960833333
current encoder az, el: 135.0 35.6416519072
az offset:  -41.9114166667
el offset: 0.0455685738542
applied az offset (start/finish):  242.188481657 242.188481657


In [108]:
166.918777778 - (178.587703879 - 53.96923784)

42.300311738999994

In [33]:
print 126.7592060546875 + 4.78998783947
print 60.6097685546875 + 3.81606304253

131.549193894
64.4258315972


In [10]:
hours = 12
sph = 30
samples = hours*sph
times = np.linspace(0, hours, samples)

azsun = []
elsun = []
azmoon = []
elmoon = []

target = 'Sun'
target2 = 'Moon'

for t in times:
    azs, els = getpointing('Sedgwick', target, datetime.datetime.utcnow()+timedelta(0, (t)*3600.))
    azsun.append(azs)
    elsun.append(els)
    #azm, elm = getpointing('Sedgwick', target2, datetime.datetime.utcnow()+timedelta(0, t*3600.))
    #azmoon.append(azm)
    #elmoon.append(elm)
    
azsun = np.asarray(azsun)
elsun = np.asarray(elsun)
#azmoon = np.asarray(azmoon)
#elmoon = np.asarray(elmoon)

        Use `lon` instead. [astropy.utils.decorators]
        Use `lat` instead. [astropy.utils.decorators]


In [11]:


figure()
plot(times,elsun, 'r-',label=target+' el')
#plot(times, elmoon, 'b-',label=target2+' el')
#title('%s el' % target)
xlabel('time (hours)')
ylabel('el (deg)')
legend()

figure()
plot(times,azsun, 'r-',label=target+' az')
#plot(times, azmoon, 'b-',label=target2+' az')
#title('%s el' % target)
xlabel('time (hours)')
ylabel('az (deg)')
legend()

#figure()
#plot(times, azsun, 'r-',label='sun')
#plot(times, azmoon, 'b-',label='moon')
#title('sun/moon az')
#xlabel('time (hours)')
#ylabel('az (deg)')
#legend()

#figure()
#dist = np.sqrt((((azsun - azmoon)**2) + ((elsun - elmoon)**2))) 
#plot(times, dist, 'r-',label='distance')
#title('sun/moon distance apart')
#xlabel('time (hours)')
#ylabel('distance (deg)')
#legend()


In [324]:
azs, els = getpointing('Greenland', 'Sun', datetime.datetime.utcnow())
print datetime.datetime.utcnow()
print azs, els

2018-08-07 11:39:29.548000
129.657305556 28.1718333333


In [323]:
print 479 % 360

119


In [61]:
ra = (5. + 34./60. + 31.94/3600.)*360./24.
dec = 22 + 52.2/3600.

azt, elt = planets.radec_to_azel(ra, dec, 'Sedgwick', datetime.datetime.utcnow())


print ra, dec
print azt, elt

83.6330833333 22.0145
98.344834452 51.9933099819


In [32]:
ra = (5. + 34./60. + 31.94/3600.)*360./24.
dec = 22 + 52.2/3600.

hours = 2
sph = 30
samples = hours*sph
times = np.linspace(0., hours, samples)

aztau = []
eltau = []

for t in times:
    #azt, elt = getpointing('Greenland', [ra, dec, 'RADEC'], datetime.datetime.utcnow()+timedelta(0, t*3600.))
    azt, elt = planets.radec_to_azel(ra, dec, 'Sedgwick', datetime.datetime.utcnow()+timedelta(0, t*3600.))
    aztau.append(azt)
    eltau.append(elt)

In [50]:
figure()
plot(times, aztau, 'g-')
title('tau a az')
xlabel('time (hours)')
ylabel('az (deg)')

figure()
plot(times, eltau, 'g-')
title('tau a el')
xlabel('time (hours)')
ylabel('el (deg)')

#figure()
#dist = np.sqrt((((azsun - aztau)**2) + ((elsun - eltau)**2))) 
#plot(times, dist, 'r-',label=' tau/sun distance')
#xlabel('time (hours)')
#ylabel('distance (deg)')

#dist2 = np.sqrt((((aztau - azmoon)**2) + ((eltau - elmoon)**2))) 
#plot(times, dist2, 'g-',label='tau/moon distance')
#xlabel('time (hours)')
#ylabel('distance (deg)')
#legend()

In [136]:
print np.asarray(aztau).min(), np.asarray(aztau).max()

256.434676159 256.483998831


In [131]:
print np.asarray(aztau).min(), np.asarray(aztau).max()

255.168941867 311.664156084


In [31]:
gps_bits=[24,48,0,73,25,49,1,74,26,50,2,75,27,51,3,76,28,52,4,77,29,53,5,78,30,54,6,79,31,55,7,80]
#gps_bits=[80,7,55,31,79,6,54,30,78,5,53,29,77,4,52,28,76,3,51,27,75,2,50,26,74,1,49,25,73,0,48,24]